In [ ]:
%pip install -U langchain_community 
%pip install -U langchain-aws

In [1]:
import os
import json
import re
from langchain_core.output_parsers.base import BaseOutputParser
from langchain_core.runnables import RunnableLambda

In [2]:
def dict_to_obj(json_str:dict, target:object):
    return target.parse_obj(json_str)


class CustJsonOuputParser(BaseOutputParser[str]): 
    def parse(self, text: str) -> str:
        # print(text)
        pattern = r"<answer>(.*?)</answer>"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            text = match.group(1)
            print(text)
            # text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
        else:
            return {'answer':"no"}    
        new_dict = json.loads(text.replace('\n','  '))
        
        return new_dict

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    


In [60]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm_sonnet = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.8,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.9,
                                "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()]
                )

llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0.8,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.9,
                                "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                  # callbacks=[StreamingStdOutCallbackHandler()]
                  )

In [45]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage

In [48]:
class Record(BaseModel):
    reason: str = Field(..., title="表示相似话题原因" )
    desc: str = Field(..., title="表示相似话题描述" )
    ids: str = Field(..., title="表示合并的话题ID列表(之间使用英文逗号分隔)")
class Results(BaseModel):
    analaysis: List[Record] = Field(
        default_factory=list,
        title="分析并合并话题之后的结果",
    )

In [6]:
fc_desc = """
You will ALWAYS follow the below guidelines when you are answering a question:
<guidelines>
- Think through the user's question, extract all data from the question and the previous conversations before creating a plan.
- Always output your thoughts within <thinking></thinking> xml tags before you respond to the user. 
- Your response must be follow the pydantic schema as:
<schema>
    {schema}
</shema>
- output your answer in json format, enclosed in tag <answer>, so that the user can use pydantic basemodel.parse_obj() to parse the json string into an object which defined as:
 <schema>
 {schema}
 </shema>
 
</guidelines>
"""

In [35]:
system = """
你是一个游戏社区运营专家，参与为游戏社区内玩家发送消息的舆情分析工作；你需要为下面玩家聊天的话题和原因，进行相似合并，并返回结果；  
规则：  
- 1.输入的数据格式为如下json，其中k表示话题ID，d表示话题内容，r表示原因:
   <example_user>
   [{{"k":"1199997647499571310","d":"玩家对复活节金蛋活动的兴趣和乐趣","r":"复活节金蛋活动带给玩家很多快乐和兴奋，让他们可以共享奖励，增强了玩家间和服务器间的互动"}},{{"k":"1199198471907987560","d":"黄金蛋活动增进了玩家间的互动与游戏热度","r":"黄金蛋活动可以在全球聊天中分享，增强了玩家间的互动和游戏的热度"}}]
   </example_user>
2.严格按照如下的xml格式输出内容到<output>,参考<example_assistant>中的内容（并且仅输出压缩后xml数据，不需要换行和缩进符，请输出合法的xml数据）
其中i表示合并后的话题对象，i中的d表示相似话题描述，i中的r表示相似话题原因，i中的ol表示合并的话题ID列表(之间使用英文逗号分隔)，话题描述、原因使用中文：
<example_assistant><i><d>玩家对复活节金蛋活动的兴趣和乐趣</d><r>复活节金蛋活动带给玩家很多快乐和兴奋，让他们可以共享奖励，增强了玩家间和服务器间的互动</r><ol>1199997647499571310,1199198471907987560</ol></i></example_assistant>
- 3.请保留原文中的符号格式
请直接输出压缩后的合法的xml内容到<output>,不要做任何解释和释义
"""

In [55]:
system = """
你是一个游戏社区运营专家，参与为游戏社区内玩家发送消息的舆情分析工作；你需要为下面玩家聊天的话题和原因，进行相似合并，并返回结果；  
规则：  
- 1.输入的数据格式为如下json，其中k表示话题ID，d表示话题内容，r表示原因:
   <example_user>
   [{{"k":"1199997647499571310","d":"玩家对复活节金蛋活动的兴趣和乐趣","r":"复活节金蛋活动带给玩家很多快乐和兴奋，让他们可以共享奖励，增强了玩家间和服务器间的互动"}},{{"k":"1199198471907987560","d":"黄金蛋活动增进了玩家间的互动与游戏热度","r":"黄金蛋活动可以在全球聊天中分享，增强了玩家间的互动和游戏的热度"}}]
   </example_user>
- 2.请分析<example_user>里面的内容。
- 3.请保留原文中的符号格式
"""+fc_desc

In [56]:
text = "[{\"k\":\"ee77e9bdd39e3bd8467aa8bd2f20a7e4\",\"d\":\"玩家对活动结束时间描述存在疑问\",\"r\":\"活动时间结束描述可能存在一些表述不当之处\"},{\"k\":\"93c60cad226fff2123b46719ed521078\",\"d\":\"玩家询问如何获得特殊奖励\",\"r\":\"玩家对获取特殊奖励的方式不太了解\"},{\"k\":\"f65c75499edaef4f1391c92344a3279a\",\"d\":\"玩家对新活动无法找到相关信息表示疑惑\",\"r\":\"新活动的推广和宣传可能还不够充分,导致部分玩家无法获取相关信息\"},{\"k\":\"32ad2bc0e97c319727093a1be36739f7\",\"d\":\"玩家决定等待其他时候再参与\",\"r\":\"活动人数过多导致体验不佳\"},{\"k\":\"43adf902729cca3b80e4097d94e0a7d9\",\"d\":\"玩家对于加入游戏有疑问\",\"r\":\"可能是新玩家想要加入游戏但不太了解加入的方式\"},{\"k\":\"6659ba49278868874dac340810634966\",\"d\":\"玩家提到开发商在某处发布了回应\",\"r\":\"开发商可能针对某个问题或建议发布了官方的回应\"},{\"k\":\"d84aeba907d7e2eb17e81dc286f7dc33\",\"d\":\"玩家遇到了准备按钮无法使用的问题\",\"r\":\"游戏中的某些功能可能存在Bug或使用限制\"},{\"k\":\"98a23e2a1813fbba5a700e642eed6fa4\",\"d\":\"玩家对游戏中一些特殊情况表示疑惑\",\"r\":\"游戏运行过程中偶尔会出现一些异常情况,玩家对此表示不解\"},{\"k\":\"3148c9d388a11de292febc1826b5d395\",\"d\":\"玩家在使用无人机装置时出现一些异常情况\",\"r\":\"无人机装置在使用过程中可能偶尔出现一些bug或异常情况\"},{\"k\":\"3ff1331e87b0c27534ccb6a993273a8a\",\"d\":\"玩家在游戏中遇到了测试、出错或延迟等问题\",\"r\":\"由于大量玩家同时在线或最新的补丁更新,可能导致网络拥塞和游戏延迟\"},{\"k\":\"1856a45ab08e2419c0e06fdbda24534b\",\"d\":\"抱歉当前游戏模式出现了网络问题,由于大量玩家参与导致测试服务器出现了故障\",\"r\":\"测试服务器受到大量玩家的热情参与影响,导致出现了故障和网络连接问题\"},{\"k\":\"4cb1f6eed66fbc82ddc79d489332a1a5\",\"d\":\"玩家对当前游戏版本存在疑问\",\"r\":\"玩家不确定当前游戏版本是否是正式版本还是测试版本\"},{\"k\":\"7bbc1fb571e8a9d30c7f2b1a975b2ad7\",\"d\":\"玩家对加入新同盟有疑问\",\"r\":\"一些新玩家可能对如何加入同盟不太了解\"},{\"k\":\"f1caadc358f96df18a81dcab54575ec7\",\"d\":\"玩家认为在防御科技树和特种兵科技树之间应该选择后者\",\"r\":\"特种兵科技树可以为小队提供更大的作战优势,相比之下防御科技树的收益较小\"},{\"k\":\"f9bf484cc0a8731a241f218026fb3b43\",\"d\":\"玩家不知道活动的具体位置\",\"r\":\"活动位置说明不够明确\"},{\"k\":\"ae3b5285c2c85afd4a7bfab34952bbe9\",\"d\":\"官方通知玩家耐心等待\",\"r\":\"网络拥堵导致的问题需要一些时间解决\"},{\"k\":\"b0e01a7fe1fd399ea38660c25fa52eba\",\"d\":\"玩家无法匹配进入游戏\",\"r\":\"由于网络故障和连接问题,玩家无法正常匹配进入游戏\"},{\"k\":\"766b02cb6521a16bc23cd8f2ffa8d858\",\"d\":\"玩家评论游戏\",\"r\":\"玩家对游戏的一些方面提出评论和建议\"},{\"k\":\"db3ea4872fc163621f753acc0635dace\",\"d\":\"玩家分享了自己的用户ID\",\"r\":\"有些玩家在讨论时分享了自己的用户ID\"},{\"k\":\"261542c9794f0a2b789dd78fa8889cbd\",\"d\":\"玩家对游戏中一些表情的使用场景存有疑问\",\"r\":\"游戏聊天频道偶尔会出现一些特殊表情,玩家对其用途不太理解\"},{\"k\":\"639053b7860bb86961306e82776b268e\",\"d\":\"玩家在战斗中遇到问题\",\"r\":\"大量玩家同时参与导致了网络拥堵和故障\"},{\"k\":\"2ceb3fb15f5e1914892ec490d3d9c45c\",\"d\":\"玩家对游戏活动是否开启在所有服务器都有疑问\",\"r\":\"不同服务器开启新活动的时间可能有所差异,玩家对此存在疑虑\"},{\"k\":\"67ee5a46b22478975437a2efed03dc91\",\"d\":\"玩家表示等待是必要的\",\"r\":\"由于玩家过多导致了延迟等问题,等待是必要的\"},{\"k\":\"97f9850b17b600bbac3ae5c9435649b2\",\"d\":\"玩家对游戏内容更新表示期待\",\"r\":\"玩家对游戏的持续更新和优化有一定的期望\"},{\"k\":\"13e61a41dad4e02f204c7d653a3cfff8\",\"d\":\"游戏开发人员忙于处理实际服务器工作\",\"r\":\"开发人员由于忙于处理实际的服务器工作,暂时无法参与游戏活动\"},{\"k\":\"3514770bee2622801295af0990206c44\",\"d\":\"讨论游戏中特殊玩家表现\",\"r\":\"讨论了一些玩家在围攻战场或者特种部队方面出色的表现\"},{\"k\":\"c274e732a80dba6d1e81354f1512024b\",\"d\":\"玩家对冬季战争活动中小队和集结的使用存在疑问\",\"r\":\"冬季战争活动中小队和集结的使用方式与沙漠风暴不同,玩家对此存在疑惑\"},{\"k\":\"73bd79c44b783e5b53948fc7aca5d860\",\"d\":\"新手玩家在学习成长\",\"r\":\"一些玩家是新手,正在学习和提升技能\"},{\"k\":\"2827e553486a5d0ac1d4a0b2de20bfb3\",\"d\":\"有玩家想要参与游戏测试\",\"r\":\"有玩家对参与游戏测试感兴趣,想要了解参与方式\"},{\"k\":\"33f84e8cf3ddc166c7346845324f6c13\",\"d\":\"玩家对一些技能或任务触发条件不太明确\",\"r\":\"游戏中部分技能或任务的触发条件描述可能不太清晰明了\"},{\"k\":\"7c99a74a3107bf613b63b35659554cbd\",\"d\":\"玩家想了解该活动是否适用于所有服务器\",\"r\":\"新活动的开放范围可能引起了部分玩家的疑问\"},{\"k\":\"d4e92b65917527b22d832ba3f5709ddd\",\"d\":\"玩家对冬日风暴战场活动的时间、参与资格以及奖励有疑问\",\"r\":\"新活动带来了许多不确定性,玩家希望了解更多细节\"},{\"k\":\"362dbbd2e43e91025f46921bf542ec78\",\"d\":\"玩家询问问题出现的原因\",\"r\":\"游戏问题的原因让玩家感到疑惑\"},{\"k\":\"a9f0277f7038263f9497b299b66d6701\",\"d\":\"玩家对活动时间安排存有疑问\",\"r\":\"活动时间长短和时间点安排等细节可能让部分玩家感到疑惑\"},{\"k\":\"de6b899a15f4de472f36e598509312b2\",\"d\":\"玩家建议退出频道等待问题解决\",\"r\":\"由于网络问题导致的故障无法立即解决\"},{\"k\":\"41e151c1e409df21105cfb57f90dc39d\",\"d\":\"玩家询问活动持续时间\",\"r\":\"活动持续时间说明不够清楚\"},{\"k\":\"7f8dec431341418463332ee42ceca387\",\"d\":\"玩家讨论软件项目部署和测试的难度\",\"r\":\"部署和测试软件项目会遇到很多难以预料的问题,需要渐进式地完善和优化\"},{\"k\":\"85e8d2ce934efdba96f508c9d4048caf\",\"d\":\"玩家对矿场内的小队布防情况提出疑问\",\"r\":\"矿场内小队布防机制可能与其他区域有所不同,引发玩家的疑惑\"},{\"k\":\"051be5d6f5c7f73b8c129315656b85b0\",\"d\":\"玩家对防守工事科技研究的要求有疑问\",\"r\":\"防守工事科技研究存在一些特殊要求,玩家不太理解其中的逻辑\"},{\"k\":\"ab74801d981e5b4b85867ee63ecff9f1\",\"d\":\"玩家希望游戏能尽快修复\",\"r\":\"玩家对游戏有热情,希望游戏能尽快修复网络故障等问题\"},{\"k\":\"94b6d73800fd4cfc77a3736a53a18156\",\"d\":\"玩家收到一些特殊代码\",\"r\":\"可能是游戏临时生成的一些特殊代码,用于问题反馈或测试\"},{\"k\":\"83769ba783743358cb7d44b0f2d4e2f4\",\"d\":\"玩家表示最近的补丁导致了一些延迟和Bug问题\",\"r\":\"最新的游戏补丁更新可能带来了一些新的问题和Bug\"},{\"k\":\"b464042c155ba2b640ae3a58e3e8ecb5\",\"d\":\"玩家对新活动的内容和体验提出了评价\",\"r\":\"新活动可能存在一些Bug和缺陷,但整体内容和体验还是不错的\"},{\"k\":\"34bb3d59cea7636d1f1979d0c325b5f6\",\"d\":\"玩家对游戏机制存在一些疑问和建议\",\"r\":\"游戏中的一些机制设置可能让部分玩家感到困惑或不太满意\"},{\"k\":\"bd4339d2412fcf5486d986d64c79068f\",\"d\":\"玩家想知道其他玩家所在的服务器\",\"r\":\"了解其他玩家的服务器可以方便沟通和游戏体验的交流\"},{\"k\":\"164a6ba2de32efa858147ab12835a0a8\",\"d\":\"玩家询问可以参与的服务器范围\",\"r\":\"玩家不太清楚该活动针对的服务器范围\"},{\"k\":\"b910bb4d5057a4233379d2ec8aad6901\",\"d\":\"玩家对活动中的某些操作存在疑问\",\"r\":\"新活动中可能有一些特殊的操作方式,导致了部分玩家的困惑\"},{\"k\":\"ac4147fdb5fe1f53fec9c1c0ee6040e0\",\"d\":\"玩家对某个活动的时间和服务器开放范围存在疑问\",\"r\":\"新活动的开放时间和服务器范围可能没有被很好地传达\"},{\"k\":\"d838563c58156b1254e644c26955cdd0\",\"d\":\"玩家对新版本的功能有疑问\",\"r\":\"新版本带来了一些新的功能变化,玩家希望了解相关信息\"},{\"k\":\"0e973e9820c73ee6a619bb178dc121af\",\"d\":\"玩家对获取游戏反馈奖励有疑问\",\"r\":\"提供反馈是否会获得相应的奖励,玩家对此存有疑虑\"},{\"k\":\"ab5e1d59c97769194676b142ff477bd6\",\"d\":\"玩家希望活动时间更短一些\",\"r\":\"玩家认为一些活动时间太长\"},{\"k\":\"2b3ac777283d9810a574fc67652fca6f\",\"d\":\"玩家想了解其他玩家的游戏ID\",\"r\":\"了解其他玩家的游戏ID可以方便加好友和交流\"},{\"k\":\"973b462ba7e4c48ddd3360e032edc106\",\"d\":\"玩家对即将到来的活动感到疑惑\",\"r\":\"有玩家对即将到来的冬季主题活动感到疑惑,因为他们当前所在地区是夏季\"},{\"k\":\"d5f13a1bdcceb8866a2988441d264cc9\",\"d\":\"一些其他话题\",\"r\":\"还有一些其他不太明确的话题\"}]"

In [57]:
gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [61]:
outparser = CustJsonOuputParser()
chain = gen_prompt | llm_haiku | outparser

In [62]:
analaysis = chain.invoke({"messages":[HumanMessage(content=text)],'schema':Results.schema_json()})


{
  "analaysis": [
    {
      "reason": "玩家对活动时间、结束时间、奖励获取等方面存在疑问",
      "desc": "玩家对活动的具体信息,如时间安排、获取奖励的方式等存在一些疑问和不确定性",
      "ids": "f9bf484cc0a8731a241f218026fb3b43,41e151c1e409df21105cfb57f90dc39d,ac4147fdb5fe1f53fec9c1c0ee6040e0,d4e92b65917527b22d832ba3f5709ddd"
    },
    {
      "reason": "一些玩家希望活动时间更短一些",
      "desc": "部分玩家认为一些活动的时间太长,希望能缩短",
      "ids": "ab5e1d59c97769194676b142ff477bd6"
    },
    {
      "reason": "部分玩家不了解活动的具体位置和开放范围",
      "desc": "玩家对新活动的开放范围和具体位置存在疑问,希望能得到更明确的说明",
      "ids": "7c99a74a3107bf613b63b35659554cbd,164a6ba2de32efa858147ab12835a0a8"
    },
    {
      "reason": "玩家对一些特殊功能、技能或任务的触发条件不太明确",
      "desc": "游戏中的某些功能、技能或任务的触发条件描述可能不够清晰,导致玩家感到疑惑",
      "ids": "33f84e8cf3ddc166c7346845324f6c13,85e8d2ce934efdba96f508c9d4048caf,051be5d6f5c7f73b8c129315656b85b0"
    },
    {
      "reason": "玩家对游戏中的某些机制设置提出疑问和建议",
      "desc": "玩家对游戏的一些机制设计提出了疑问和建议,希望得到优化",
      "ids": "34bb3d59cea7636d1f1979d0c325b5f6"
    },
    {
      "reason": "新手玩家在学习和成长过

In [63]:
analaysis

{'analaysis': [{'reason': '玩家对活动时间、结束时间、奖励获取等方面存在疑问',
   'desc': '玩家对活动的具体信息,如时间安排、获取奖励的方式等存在一些疑问和不确定性',
   'ids': 'f9bf484cc0a8731a241f218026fb3b43,41e151c1e409df21105cfb57f90dc39d,ac4147fdb5fe1f53fec9c1c0ee6040e0,d4e92b65917527b22d832ba3f5709ddd'},
  {'reason': '一些玩家希望活动时间更短一些',
   'desc': '部分玩家认为一些活动的时间太长,希望能缩短',
   'ids': 'ab5e1d59c97769194676b142ff477bd6'},
  {'reason': '部分玩家不了解活动的具体位置和开放范围',
   'desc': '玩家对新活动的开放范围和具体位置存在疑问,希望能得到更明确的说明',
   'ids': '7c99a74a3107bf613b63b35659554cbd,164a6ba2de32efa858147ab12835a0a8'},
  {'reason': '玩家对一些特殊功能、技能或任务的触发条件不太明确',
   'desc': '游戏中的某些功能、技能或任务的触发条件描述可能不够清晰,导致玩家感到疑惑',
   'ids': '33f84e8cf3ddc166c7346845324f6c13,85e8d2ce934efdba96f508c9d4048caf,051be5d6f5c7f73b8c129315656b85b0'},
  {'reason': '玩家对游戏中的某些机制设置提出疑问和建议',
   'desc': '玩家对游戏的一些机制设计提出了疑问和建议,希望得到优化',
   'ids': '34bb3d59cea7636d1f1979d0c325b5f6'},
  {'reason': '新手玩家在学习和成长过程中遇到一些问题',
   'desc': '一些新手玩家正在学习游戏并提升自己的技能,可能会遇到一些困惑',
   'ids': '73bd79c44b783e5b53948fc7aca5d860'},
  {'reason': 